## Node 정의

이진 검색트리의 Node는 다음과 같은 멤버 변수를 갖는다
* parent: 노드의 parent node의 레퍼런스, parent가 None이면, root node로 간주한다.
* left  : 노드의 왼쪽 child node의 레퍼런스
* right : 노드의 오른쪽 child node의 레퍼런스
* key   : 검색 키 값

또한, 레드블랙트리를 지원하기 위하여 다음과 같은 멤버 변수를 가질 수 있다.
* color : 블랙=1, 레드=0
* nullNode : 레드블랙트리의 leaf node의 child node는 NULL 이 아니라 특별히 정의된 공백 노드의 레퍼런스이다. nullNode의 값이 1이면, 공백 노드를 의미한다. 공백 노드는 블랙 노드이며, 두 child는 모두 None이다.



In [ ]:
class Node:
    def __init__(self, key, nullNode=0):
        self.parent = None
        self.key = key
        self.left = None    #None or empty leaf node for red black tree
        self.right = None   
        
        #these items are for a red black tree
        self.color = 0      #black(1), red(0)
        self.nullNode = nullNode #if it is 1, it means a Null leaf node of a red black tree
    
    def addChild(self, leftOrRight, childNode):
        
        if leftOrRight == 0: #left child
            self.left = childNode 
            childNode.parent = self
            
        elif leftOrRight == 1: #right child            
            self.right = childNode 
            childNode.parent = self

        else :
            return 1
        return 0
    
    
    def readKey(self):
        return self.key
    
    def writeKey(self, key):
        self.key = key
        
        


## BST의 노드 검색

주어진 node를 root로 하는 이진 검색트리에서 key값을 갖는 노드를 검색하여 리턴한다.
* result = 0 이면, key 값을 갖는 노드를 찾음
* result = 1 이면, key 값을 갖는 노드를 찾을 수 없음

In [ ]:
def binarySearchTreeFindNode(node: Node, key) :
    
    if node == None or key == None:
        return 1, None
    
    if node.key == key:
        result = 0    
    elif node.key < key :
        result, node = binarySearchTreeFindNode(node.right, key)
    else :
        result, node = binarySearchTreeFindNode(node.left, key)
            
    return result, node

 ## BST에서 새로운 노드 추가
 
root와 key 값이 주어지면, 주어진 key 값을 갖는 새로운 노드를 생성하여 root를 참조하여 트리에 새로운 노드를 삽입한다.
treeType은 레드블랙트리를 지원하기 위한 파라메타로, treeType=1이면, 레드블랙트리에 노드를 삽입하기 위해 함수가 호출되었음을 의미한다.

root가 None이면, 트리를 새로 생성하는 것으로 간주하고 root를 생성한 후 종료한다.

root가 None이 아니면, key 값을 참조하여 적절한 leaf를 찾아 노드를 삽입한다.

함수 호출 후, result = 0 이 리턴되면 정상종료된 것을 의미하며, 0 보다 큰 값이 리턴되면 오류가 발생한 것으로 간주한다.

In [ ]:
def _binarySearchTreeInsertNode(parent: Node, key, treeType) -> (int, Node):    
     
    if parent == None or parent.nullNode == 1:  # if nullNode==1, it is a empty leaf node for red black tree
        return 1, None
    
    # to check whether input type of 'parent' is Node
    # if isinstance(parent,Node) == False:
    #    return 2, None    
    
    node=None
    
    if key < parent.key:
        result, node =  _binarySearchTreeInsertNode(parent.left, key, treeType)
        if result == 1:
            node = Node(key)
            parent.addChild(0, node) #add a left child to a parent node
            
    else:
        result, node =  _binarySearchTreeInsertNode(parent.right, key, treeType)
        if result == 1:
            node = Node(key)
            parent.addChild(1, node) #add a right child to a parent node
    
    #if a tree is a red black tree, a leaf node is an empty leaf node, not a node which does not any child node
    if treeType == 1:
        emptyNode_left = Node(0,1)  #key is 0 and nullNode is 1
        emptyNode_left.parent=node
        emptyNode_left.color=1
        emptyNode_right = Node(0,1)
        emptyNode_right.parent=node
        emptyNode_right.color=1
            
        node.left = emptyNode_left
        node.right = emptyNode_right        
            
    return 0, node


def binarySearchTreeInsertNode(root: Node, key, treeType=0) -> (int, Node):  
    
    if root == None: #generate a root
        node = Node(key, 0)  
        if treeType == 1:
            emptyNode_left = Node(0,1) #key is 0 and nullNode is 1
            emptyNode_left.parent=node
            emptyNode_left.color=1
            emptyNode_right = Node(0,1)
            emptyNode_right.parent=node
            emptyNode_right.color=1
            
            node.color = 1  #a root is a black node
            node.left = emptyNode_left
            node.right = emptyNode_right 
            
        return 0, node
    else:
        result, node = _binarySearchTreeInsertNode(root, key, treeType)
        return result, node 


## BST에서 노드 삭제

주어진 키 값을 이용하여 노드를 탐색한 후, 탐색된 노드를 삭제한다.

In [ ]:
def _deleteNode(node):
    if node.left == None and node.right == None:
        return None
    
    elif node.left != None and node.right == None:
        return node.left
    
    elif node.left == None and node.right != None:
        return node.right
    
    else:
        target = node.right 
        while target.left != None:
            parent = target
            target = target.left
        
        #delete target
        node.key = target.key
        
        if target == node.right:
            node.right = target.right
        else:
            parent.right = target.right
        return node 

# it finds a node with removeNode.key and then remove the node from tree with the root    
def binarySearchTreeDeleteNode(root:Node, key):
    
    result, removeNode = binarySearchTreeFindNode(root, key)
    
    if result == 0:
        if removeNode.key == root.key:
            root = _deleteNode(removeNode)
            
        elif removeNode.parent.left != None and removeNode.parent.left.key == removeNode.key :
            removeNode.parent.left = _deleteNode(removeNode)
        
        else:
            removeNode.parent.right = _deleteNode(removeNode)
    
    return result, removeNode, root

## BST Test

In [ ]:
result, root=binarySearchTreeInsertNode(None,2)
result, node=binarySearchTreeInsertNode(root,4)
result, node=binarySearchTreeInsertNode(root,5)
result, node=binarySearchTreeInsertNode(root,1)
print(root)
node = binarySearchTreeFindNode(root, 5)
print(node)
result, remove, root = binarySearchTreeDeleteNode(root, 2)
print(root)